# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
!pip install xgboost


You should consider upgrading via the 'c:\users\mbadran\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import ast
# magic word for producing visualizations in notebook
%matplotlib inline
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.decomposition import PCA 
import pprint
from sklearn.cluster import KMeans

KeyboardInterrupt: 

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
# load in the data
azdias_df = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
customers_df = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')
attributes_df = pd.read_excel("DIAS Information Levels - Attributes 2017.xlsx", header = 1, index_col=[1,2]).reset_index()
values_df = pd.read_excel('DIAS Attributes - Values 2017.xlsx', header=1, index_col=[1,2]).reset_index()

In [ ]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!

# Set pandas display to be able to scroll through all columns and rows
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows',200)

In [ ]:
azdias_df.head()

In [ ]:
azdias_df.info()

In [ ]:
azdias_df.describe()

In [ ]:
azdias_df.drop_duplicates()

In [ ]:
customers_df.info()

In [ ]:
customers_df.head()

In [ ]:
customers_df.describe()

In [ ]:
print(customers_df.shape)
customers_df.head()

In [ ]:
customers_df.drop_duplicates()

In [ ]:
np.intersect1d(azdias_df.columns.values, customers_df.columns.values).shape

In [ ]:
only_attr_customers = np.setdiff1d(customers_df.columns.values, azdias_df.columns.values)
only_attr_customers

In [ ]:
plt.subplots(1, 2, figsize = [12, 5])

plt.subplot(1, 2, 1)
customers_df.CUSTOMER_GROUP.value_counts().plot.bar(rot=0)

plt.subplot(1, 2, 2)
customers_df.CUSTOMER_GROUP.value_counts().plot.pie(autopct='%1.f%%', startangle = 90, counterclock = False).set_ylabel('')

plt.suptitle('CUSTOMER_GROUP Distribution', fontsize=16)
plt.show()

In [ ]:
plt.subplots(1, 2, figsize = [12, 5])

plt.subplot(1, 2, 1)
customers_df.ONLINE_PURCHASE.value_counts().plot.bar(rot=0)

plt.subplot(1, 2, 2)
customers_df.ONLINE_PURCHASE.value_counts().plot.pie(autopct='%1.f%%', startangle = 90, counterclock = False).set_ylabel('')

plt.suptitle('ONLINE_PURCHASE Distribution', fontsize=16)
plt.show()

In [ ]:
plt.subplots(1, 2, figsize = [12, 5])

plt.subplot(1, 2, 1)
customers_df.PRODUCT_GROUP.value_counts().plot.bar(rot=0)

plt.subplot(1, 2, 2)
customers_df.PRODUCT_GROUP.value_counts().plot.pie(autopct='%1.f%%', startangle = 90, counterclock = False).set_ylabel('')

plt.suptitle('PRODUCT_GROUP Distribution', fontsize=16)
plt.show()

In [ ]:
# attributes dataset

attributes_df.head()

In [ ]:
attributes_df.info()

In [ ]:
values_df.head()

In [ ]:
values_df.info()

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## General Population Analysis

In [ ]:
azdias_df.head()

customers_df.head()

In [ ]:
a_feats = pd.DataFrame(columns=['unique_vals', 'perc_is_na'])

for col in azdias_df.columns:
    vals = int(len(azdias_df[col].unique()))
    perc = round((np.sum(azdias_df[col].isna()) / azdias_df.shape[0])*100, 2)
    a_feats.loc[col] = [vals, perc]
    
a_feats = a_feats.sort_values(['perc_is_na', 'unique_vals'], ascending=False)

In [ ]:
a_feats.head()

## Customers Analysis

In [ ]:
c_feats = pd.DataFrame(columns=['unique_vals', 'perc_is_na'])

for col in customers_df.columns:
    vals = int(len(customers_df[col].unique()))
    perc = round((np.sum(customers_df[col].isna()) / customers_df.shape[0])*100, 2)
    c_feats.loc[col] = [vals, perc]
    
c_feats = c_feats.sort_values(['perc_is_na', 'unique_vals'], ascending=False)

In [ ]:
c_feats.head(5)

In [ ]:
unique_cols = list(set(customers_df.columns.values) - set(azdias_df.columns.values))

print("Rows in costumer set that are not in azdias: " + str(unique_cols) + "\n")

for unique_col in unique_cols:
    print("Unique values for column: " + str(unique_col) + " - " + str(set(customers_df[unique_col].values)))
    
print()
    
for unique_col in unique_cols:
    counts = customers_df[unique_col].value_counts()
    sum_c = np.sum(customers_df[unique_col].value_counts())
    print(str(unique_col))
    
    for idx in counts.index:
        print(str(idx) + ": " + str(round(counts[idx] / sum_c ,3) * 100) + "%")
        
    print(25*"=")

In [ ]:
cleaned_customers = customers_df.copy()
cleaned_customers = cleaned_customers[customers_df['GEBURTSJAHR'] != 0]

cleaned_azdias = azdias_df.copy()
cleaned_azdias = cleaned_azdias[azdias_df['GEBURTSJAHR'] != 0]

## Compare General public population to Customers


In [ ]:
compare_az_cus = pd.DataFrame(columns=['a_max', 'c_max', 'a_min', 'c_min', 'a_mean', 'c_mean', 'a_median', 'c_median', 'mean_diff'])

common_cols = np.intersect1d(azdias_df.columns, customers_df.columns)

for col in common_cols:
    if(customers_df[col].dtype == int):
        
        a_max = cleaned_azdias[col].max()
        c_max = cleaned_customers[col].max()
        a_min = cleaned_azdias[col].min()
        c_min = cleaned_customers[col].min()
        a_mean = cleaned_azdias[col].mean()
        c_mean = cleaned_customers[col].mean()
        a_median = cleaned_azdias[col].median()
        c_median = cleaned_customers[col].median()
        diff = np.abs(a_mean - c_mean)
        
        compare_az_cus.loc[col] = a_max, c_max,  a_min, c_min, a_mean, c_mean, a_median, c_median, diff
        
    else:  
        try:
            a_max = cleaned_azdias[col].max()
            c_max = cleaned_customers[col].max()
            a_min = cleaned_azdias[col].min()
            c_min = cleaned_customers[col].min()

            compare_az_cus.loc[col] = a_max, c_max,  a_min, c_min, np.nan, np.nan, np.nan, np.nan, np.nan
        except:
            
            compare_az_cus.loc[col] = np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

In [ ]:
compare = compare_az_cus[~compare_az_cus['c_mean'].isna()].loc[compare_az_cus['mean_diff'] > 0.5].sort_values('mean_diff', ascending=False).drop('LNR', axis=0)

## Heatmap for the Customer correlation

In [ ]:
additional_cols = np.array(['FINANZ_UNAUFFAELLIGER', 'D19_GESAMT_ONLINE_DATUM',
       'D19_VERSAND_OFFLINE_DATUM', 'SEMIO_TRADV', 'ALTERSKATEGORIE_GROB'])

In [ ]:
columns = np.concatenate((np.array(compare.index[:15]), additional_cols, np.array(unique_cols)), axis=0)

heatmap_c = customers_df[columns]
heatmap_c = heatmap_c[heatmap_c['GEBURTSJAHR'] != 0]

heatmap_c['PRODUCT_GROUP'] = heatmap_c['PRODUCT_GROUP'].factorize()[0]
heatmap_c['CUSTOMER_GROUP'] = heatmap_c['CUSTOMER_GROUP'].factorize()[0]

In [ ]:
plt.figure(figsize = (16,10))
ax = sns.heatmap(heatmap_c.corr(), cmap='RdYlGn_r', annot=True, linewidths=.5)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

## Heatmap for the Azdias correlation

In [ ]:
columns = np.concatenate((compare.index[:15], additional_cols), axis=0)
heatmap_a = azdias_df[columns]
heatmap_a = heatmap_a[heatmap_a['GEBURTSJAHR'] != 0]

In [ ]:
plt.figure(figsize = (16,10))
ax = sns.heatmap(heatmap_a.corr(), cmap='RdYlGn_r', annot=True, linewidths=.5)
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

## Plot differences between General Population and Customers

## Ages and teen-years

In [ ]:
cleaned_customers[['GEBURTSJAHR', 'PRAEGENDE_JUGENDJAHRE', 'AGER_TYP']].hist(figsize = (15,5))
plt.show()

In [ ]:
cleaned_azdias[['GEBURTSJAHR', 'PRAEGENDE_JUGENDJAHRE', 'AGER_TYP']].hist(figsize = (15,5))
plt.show()

## Money Savings

In [ ]:
cleaned_customers[['FINANZ_MINIMALIST', 'FINANZ_VORSORGER', 'FINANZ_SPARER', 'FINANZ_ANLEGER']].hist(figsize = (15,5))
plt.show()

In [ ]:
cleaned_azdias[['FINANZ_MINIMALIST', 'FINANZ_VORSORGER', 'FINANZ_SPARER', 'FINANZ_ANLEGER']].hist(figsize = (15,5))
plt.show()

## Affinity mindsets

In [ ]:
cleaned_customers[['SEMIO_LUST', 'SEMIO_RAT', 'SEMIO_PFLICHT', 'SEMIO_TRADV']].hist(figsize = (15,5))
plt.show()

In [ ]:
cleaned_azdias[['SEMIO_LUST', 'SEMIO_RAT', 'SEMIO_PFLICHT', 'SEMIO_TRADV']].hist(figsize = (15,5))
plt.show()

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

In [ ]:
# machine learning
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble
from sklearn import preprocessing

from sklearn.ensemble import VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier ,RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier ,Perceptron
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
#from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV,cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.dummy import DummyClassifier

import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score ,classification_report, precision_recall_curve, roc_curve, roc_auc_score ,    confusion_matrix

In [ ]:
X = mailout_train[np.concatenate((compare.index, additional_cols))]
y = mailout_train['RESPONSE']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

print("X_train: " + str(X_train.shape))
print("y_train: " + str(y_train.shape))
print("X_test: " + str(X_test.shape))
print("y_test: " + str(y_test.shape))

In [ ]:
# Perform pre-processing to scale numeric features
scale = preprocessing.StandardScaler()
X_train = scale.fit_transform(X_train)
# Test features are scaled using the scaler computed for the training features
X_test = scale.transform(X_test)

## Model Building

In [ ]:
def evaluate(name,model, X_train, X_test, y_train, y_test,high_accuracy_score,best_model):
    model.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    print("TRAINIG RESULTS: \n===============================")
#    clf_report = pd.DataFrame(classification_report(y_train, y_train_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_train, y_train_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_train, y_train_pred):.4f}")
#    print(f"CLASSIFICATION REPORT:\n{clf_report}")

    print("TESTING RESULTS: \n===============================")
#    clf_report = pd.DataFrame(classification_report(y_test, y_test_pred, output_dict=True))
    print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_test_pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_test_pred):.4f}")
#    print(f"CLASSIFICATION REPORT:\n{clf_report}")
    
    
    scores = cross_val_score(model,  X_test, y_test, cv=3, scoring='f1')
    print(scores)
    print(f"{name} f1_score: {scores.mean() * 100:.2f}% +/- ({scores.std() * 100:.2f})")

#     disp = plot_confusion_matrix(model, X_test, y_test, 
#                              cmap='Blues', values_format='d', 
#                              display_labels=['Stay', 'Churn'])
    
#     disp = plot_roc_curve(model, X_test, y_test)
    print(f"high_accuracy_score:{high_accuracy_score}")

    if(accuracy_score(y_test, y_test_pred) > high_accuracy_score ):
        high_accuracy_score = accuracy_score(y_test, y_test_pred)
#         submission = pd.DataFrame({
#         "PassengerId": test_df["PassengerId"],
#         "Survived": y_test_pred
#         })
#         submission.to_csv('../working/submit.csv', index=False)
#         print('File predicted Created')
        print(f"high_accuracy_score:{high_accuracy_score}")
        best_model = model

    return high_accuracy_score,accuracy_score(y_test, y_test_pred),best_model

In [ ]:
classifiers = {
    "Stochastic Gradient Descent": SGDClassifier(random_state=42), 
    "Logistic Regression"        : LogisticRegression(random_state=42), 
#    "Support Vector Machines"    : SVC(gamma=2, C=1,probability=True), 
    "Linear SVC"                 : LinearSVC(loss="hinge"),
    "Decision Tree"              : DecisionTreeClassifier(max_depth=5), 
    "Random Forest"              : RandomForestClassifier(n_estimators=110,oob_score=True, random_state=1, 
                                         warm_start=True, n_jobs=-1), 
    "Dummy"                      : DummyClassifier(strategy='uniform', random_state=2),
#    "k-Nearest Neighbors"        : KNeighborsClassifier(n_neighbors=46), 
    "Perceptron"                 : Perceptron(), 
    "Bagging Classifier"         : BaggingClassifier(), 
    "Bagging Classifier 1"       : BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=5), n_estimators=50, max_samples=0.6, n_jobs=-1), 
    "Bagging Classifier 2"       : BaggingClassifier(base_estimator=LogisticRegression(random_state=42), n_estimators=50, max_samples=0.6, n_jobs=-1), 
    "Neural Net"                 : MLPClassifier(alpha=1), 
    "AdaBoost"                   : AdaBoostClassifier(),
    "AdaBoost 1"                 : AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=5), n_estimators=50, algorithm="SAMME", random_state=42),
    "AdaBoost 2"                 : AdaBoostClassifier(base_estimator= SGDClassifier(random_state=42), n_estimators=50, algorithm="SAMME", random_state=42),
    "Gaussian Naive Bayes"       : GaussianNB(), 
#    "QDA"                        : QuadraticDiscriminantAnalysis(),
#    "Gaussian Proc"              : GaussianProcessClassifier(1.0 * RBF(1.0)),
    "Extra Trees"                : ExtraTreesClassifier(n_estimators=50, max_depth=10, max_features=0.7, random_state=42, n_jobs=-1), 
    "Gradient Boosting"          : GradientBoostingClassifier(max_depth=10, random_state=42)
#    ,
#    "xgboost"                    : XGBClassifier(max_depth=10, random_state=42, n_jobs=-1)
}

In [ ]:
from time import perf_counter

models = pd.DataFrame(columns=['Model', 'Score', 'time'])
nfast = 9      # Not running the Gaussian Process, because it's very very slow method
head = list(classifiers.items())[:]
high_accuracy_score = 0
best_model = None
for name, classifier in head:
    start = perf_counter()
    high_accuracy_score , acc_score , best_model = evaluate(name,classifier, X_train, X_test, y_train, y_test,high_accuracy_score,best_model)
    end = perf_counter()
    execution_time = (end - start)
    models = models.append({'Model': name, 'Score': acc_score, 'time': execution_time}, ignore_index=True)  
a
models.sort_values(by='Score', ascending=False)

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')

In [ ]:
mailout_test.shape

In [ ]:
X = mailout_test[np.concatenate((compare.index, additional_cols))]

print("X: " + str(X.shape))

In [ ]:
y_pred = best_model.predict(X)

In [ ]:
pred_df = pd.DataFrame({'LNR': mailout_test['LNR'], 'RESPONSE': y_pred})
pred_df.head()

In [ ]:
pred_df.to_csv('Submit.csv', index=False)